In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
import tensorflow as tf

# Reduce TensorFlow log noise
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

# Reset TensorFlow internal state
tf.keras.backend.clear_session()

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(f"Found GPU: {gpu}")
        # Enable memory growth
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPUs found!")


In [47]:
# Model and training parameters
epochs = 1
dataset = 10
how = "normal"
action = "eval"
threshold = 0.5
contrast = 1
weight = 7.0
distort = False
batch_size = 32

# Hyperparameters
epsilon = 1e-8
epochs_per_decay = 5
decay_factor = 0.80
staircase = True
lamC = 0.00001
lamF = 0.00250

# Training options
dropout = True
fc_dropout_rate = 0.5
conv_dropout_rate = 0.001
pool_dropout_rate = 0.1
num_classes = 2

In [48]:
def get_batches(X, y, batch_size, filenames=None, distort=False):
    shuffled_idx = np.arange(len(y))
    np.random.shuffle(shuffled_idx)
    i, h, w, c = X.shape

    for i in range(0, len(y), batch_size):
        batch_idx = shuffled_idx[i:i + batch_size]
        X_return = X[batch_idx]

        if distort:
            coin = np.random.binomial(1, 0.5, size=None)
            if coin:
                X_return = X_return[..., ::-1, :]

        if filenames is None:
            yield X_return, y[batch_idx]
        else:
            yield X_return, y[batch_idx], filenames[batch_idx]

In [49]:
def _scale_input_data(X, contrast=None, mu=104.1353, scale=255.0):
    if contrast and contrast != 1.0:
        X_adj = tf.image.adjust_contrast(X, contrast)
    else:
        X_adj = X

    X_adj = tf.cast(X_adj, dtype=tf.float32)
    X_adj = tf.subtract(X_adj, mu)
    X_adj = tf.divide(X_adj, scale)
    return X_adj

In [50]:
def augment(images, labels, horizontal_flip=False, vertical_flip=False, augment_labels=False, mixup=0):
    images = tf.cast(images, tf.float32)
    
    if horizontal_flip:
        images = tf.image.random_flip_left_right(images)
    if vertical_flip:
        images = tf.image.random_flip_up_down(images)
        
    if mixup > 0:
        beta = tf.random.beta([tf.shape(images)[0]], mixup, mixup)
        ll = tf.reshape(beta, [-1, 1, 1, 1])
        shuffled_images = tf.roll(images, shift=1, axis=0)
        images = ll * images + (1 - ll) * shuffled_images
        
        if augment_labels:
            labels = beta * labels + (1 - beta) * tf.roll(labels, shift=1, axis=0)
            
    return images, labels

In [51]:
class MammographyCNN(tf.keras.Model):
    def __init__(self, num_classes=2, dropout=True, fc_dropout_rate=0.5, 
                 conv_dropout_rate=0.001, pool_dropout_rate=0.1):
        super(MammographyCNN, self).__init__()
        
        self.dropout = dropout
        self.fc_dropout_rate = fc_dropout_rate
        self.conv_dropout_rate = conv_dropout_rate
        self.pool_dropout_rate = pool_dropout_rate
        
        # Conv1 block
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same',
                                          kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv11 = tf.keras.layers.Conv2D(32, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn11 = tf.keras.layers.BatchNormalization()
        self.conv12 = tf.keras.layers.Conv2D(32, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn12 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')
        
        # Conv2 block
        self.conv21 = tf.keras.layers.Conv2D(64, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn21 = tf.keras.layers.BatchNormalization()
        self.conv22 = tf.keras.layers.Conv2D(64, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn22 = tf.keras.layers.BatchNormalization()
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2), padding='same')
        
        # Global Average Pooling to handle spatial dimensions
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()

        # Fully connected layers
        self.fc1 = tf.keras.layers.Dense(2048, activation='relu')
        self.fc2 = tf.keras.layers.Dense(2048, activation='relu')
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        x = inputs  # Use inputs directly
        
        # Conv1 block
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv11(x)
        x = self.bn11(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv12(x)
        x = self.bn12(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.pool1(x)
        if self.dropout and training:
            x = tf.nn.dropout(x, rate=self.pool_dropout_rate)
        
        # Conv2 block
        x = self.conv21(x)
        x = self.bn21(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv22(x)
        x = self.bn22(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.pool2(x)
        if self.dropout and training:
            x = tf.nn.dropout(x, rate=self.pool_dropout_rate)

        # Global Average Pooling
        x = self.global_pool(x)

        # Fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

    def __init__(self, num_classes=2, dropout=True, fc_dropout_rate=0.5, 
                 conv_dropout_rate=0.001, pool_dropout_rate=0.1):
        super(MammographyCNN, self).__init__()
        
        self.dropout = dropout
        self.fc_dropout_rate = fc_dropout_rate
        self.conv_dropout_rate = conv_dropout_rate
        self.pool_dropout_rate = pool_dropout_rate
        
        # Conv1 block
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same',
                                          kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv11 = tf.keras.layers.Conv2D(32, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn11 = tf.keras.layers.BatchNormalization()
        self.conv12 = tf.keras.layers.Conv2D(32, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn12 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')
        
        # Conv2 block
        self.conv21 = tf.keras.layers.Conv2D(64, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn21 = tf.keras.layers.BatchNormalization()
        self.conv22 = tf.keras.layers.Conv2D(64, (3, 3), padding='same',
                                           kernel_regularizer=tf.keras.regularizers.l2(lamC))
        self.bn22 = tf.keras.layers.BatchNormalization()
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2), padding='same')
        
        # Additional blocks as per original architecture...
        # FC layers
        self.fc1 = tf.keras.layers.Conv2D(2048, (5, 5), strides=(5, 5), padding='valid')
        self.bn_fc1 = tf.keras.layers.BatchNormalization()
        self.fc2 = tf.keras.layers.Conv2D(2048, (1, 1), padding='valid')
        self.bn_fc2 = tf.keras.layers.BatchNormalization()
        self.fc3 = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=False):
        x = inputs  # Use inputs directly
        
        # Conv1 block
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv11(x)
        x = self.bn11(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv12(x)
        x = self.bn12(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.pool1(x)
        if self.dropout and training:
            x = tf.nn.dropout(x, rate=self.pool_dropout_rate)
        
        # Conv2 block
        x = self.conv21(x)
        x = self.bn21(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv22(x)
        x = self.bn22(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.pool2(x)
        if self.dropout and training:
            x = tf.nn.dropout(x, rate=self.pool_dropout_rate)
            
        # FC layers
        x = self.fc1(x)
        x = self.bn_fc1(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.fc2(x)
        x = self.bn_fc2(x, training=training)
        x = tf.nn.relu(x)
        
        x = tf.squeeze(x)
        x = self.fc3(x)
        
        return x

In [52]:
def get_training_data(what=10):
    if what == 10:
        train_path_10 = "data/training10_0.tfrecords"
        train_path_11 = "data/training10_1.tfrecords"
        train_path_12 = "data/training10_2.tfrecords"
        train_path_13 = "data/training10_3.tfrecords"
        
        train_files = [train_path_10, train_path_11, train_path_12, train_path_13]
        total_records = 44712
    return train_files, total_records

def get_test_data(what=10):
    test_files = "data/training10_4.tfrecords"
    return [test_files], 11178

In [53]:
def create_dataset(filenames, batch_size, is_training=False):
    feature_description = {
        'label': tf.io.FixedLenFeature([], tf.int64),
        'label_normal': tf.io.FixedLenFeature([], tf.int64),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    
    def _parse_function(example_proto):
        parsed_features = tf.io.parse_single_example(example_proto, feature_description)
        image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
        image = tf.reshape(image, [299, 299, 1])
        image = tf.cast(image, tf.float32)
        return image, parsed_features['label_normal']
    
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(_parse_function)
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

In [54]:
@tf.function
def train_step(model, images, labels, optimizer, loss_fn):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        weights = tf.multiply(weight, tf.cast(tf.greater(labels, 0), tf.float32)) + 1
        loss = loss_fn(labels, predictions, sample_weight=weights)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, predictions

In [55]:
# Setup
train_files, total_records = get_training_data(dataset)
test_files, test_records = get_test_data(dataset)
steps_per_epoch = int(total_records / batch_size)

# Create model and optimizer
model = MammographyCNN(num_classes=num_classes, dropout=dropout,
                      fc_dropout_rate=fc_dropout_rate,
                      conv_dropout_rate=conv_dropout_rate,
                      pool_dropout_rate=pool_dropout_rate)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001, decay_steps=steps_per_epoch*epochs_per_decay,
    decay_rate=decay_factor, staircase=staircase)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Metrics
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')
test_precision = tf.keras.metrics.Precision()
test_recall = tf.keras.metrics.Recall()

# Checkpointing
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, './model', max_to_keep=3)

# Load checkpoints if they exist
if os.path.exists('./model'):
    print("Restoring from checkpoint...")
    checkpoint.restore(checkpoint_manager.latest_checkpoint)

# Create datasets
train_dataset = create_dataset(train_files, batch_size, is_training=True)
test_dataset = create_dataset(test_files, batch_size)

In [ ]:
test_path = "data/training10_4.tfrecords"
print(f"File exists: {os.path.exists(test_path)}")

# List contents of /data
print("\nContents of /data:")
print(os.listdir('data/'))

In [ ]:
# Training/evaluation loop
checkpoint_every = 5  # Define checkpoint interval
print(model.summary())

# Training/evaluation loop
if action == "train":
    print("Training model...")
    for epoch in range(epochs):
        train_loss.reset_state()  # Correct method
        train_accuracy.reset_state()  # Correct method

        for images, labels in train_dataset:
            # Scale input images if required
            scaled_images = _scale_input_data(images)

            # Perform a training step
            loss, predictions = train_step(model, scaled_images, labels, optimizer, loss_fn)

            # Convert predictions to class indices if using sparse labels
            predictions = tf.argmax(predictions, axis=-1)

            # Debugging: Print shapes
            print(f"Train Images Shape: {scaled_images.shape}, Labels Shape: {labels.shape}, Predictions Shape: {predictions.shape}")

            # Update metrics
            train_loss.update_state(loss)
            train_accuracy.update_state(labels, predictions)

        # Save model checkpoint every few epochs
        if epoch % checkpoint_every == 0:
            checkpoint_manager.save()

        # Print epoch summary
        print(f"Epoch {epoch + 1}, Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()}")
else:  # Evaluation
    print("Evaluating model...")
    test_accuracy.reset_state()
    test_precision.reset_state()
    test_recall.reset_state()

    for test_images, test_labels in test_dataset:
        # Scale input images
        scaled_images = _scale_input_data(test_images)

        # Get predictions (now in [batch_size, num_classes] shape)
        predictions = model(scaled_images, training=False)

        # Debugging: Print shapes and examples
        print(f"Test Images Shape: {scaled_images.shape}")
        print(f"Test Labels Shape: {test_labels.shape}, Predictions Shape: {predictions.shape}")

        # Update metrics (no need to apply tf.argmax for SparseCategoricalAccuracy)
        test_accuracy.update_state(test_labels, predictions)
        test_precision.update_state(test_labels, tf.argmax(predictions, axis=-1))  # Precision needs class indices
        test_recall.update_state(test_labels, tf.argmax(predictions, axis=-1))  # Recall needs class indices



    # Print evaluation results
    print(f"Test Accuracy: {test_accuracy.result().numpy():.4f}")
    print(f"Test Precision: {test_precision.result().numpy():.4f}")
    print(f"Test Recall: {test_recall.result().numpy():.4f}")
